**Notes:**

1. *I have collected the messaging data came in csv and txt files.*

2. *These files are loaded into postgreSQL databases into to different table as per the months.*

3. *All of these database tables are stored in a single database source which will be helpful in the next step-2: ETL_loading_data_from_SQL_and_text_data_processing*

### Database server creations - PostgreSQL

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS uohyd_pgdaiml_7726_sc;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE uohyd_pgdaiml_7726_sc;'

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import pandas as pd
import os
import random, string
from sqlalchemy import inspect

In [ ]:
%env TFIO_DEMO_DATABASE_NAME=uohyd_pgdaiml_7726_sc
%env TFIO_DEMO_DATABASE_HOST=localhost
%env TFIO_DEMO_DATABASE_PORT=5432
%env TFIO_DEMO_DATABASE_USER=postgres
%env TFIO_DEMO_DATABASE_PASS=postgres

In [ ]:
endpoint="postgresql://{}:{}@{}?port={}&dbname={}".format(
    os.environ['TFIO_DEMO_DATABASE_USER'],
    os.environ['TFIO_DEMO_DATABASE_PASS'],
    os.environ['TFIO_DEMO_DATABASE_HOST'],
    os.environ['TFIO_DEMO_DATABASE_PORT'],
    os.environ['TFIO_DEMO_DATABASE_NAME'],
)

In [ ]:
def my_fingerprint_id_create():
    msg_id = ''.join(random.choices(string.ascii_letters + string.digits, k=12))
    return msg_id

In [ ]:
def my_data_dict_creations(dfs):
    msg_dict = dfs.to_dict('records')
    msg_id_dict = [ dict({'message_id': my_fingerprint_id_create()})for _ in range(len(msg_dict))]
    datos = zip(msg_id_dict,msg_dict)
    return datos

In [ ]:
my_psql_engine = create_engine(endpoint)

### Data Importing from csv and txt files

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/PG_Diploma_AI_ML_2021_UOHYD/PGDAIML_Project_Spam_Clustering/datos/my_test_messaging_set - Sheet1.csv')

In [ ]:
df_1 = pd.read_csv('/content/drive/MyDrive/PG_Diploma_AI_ML_2021_UOHYD/PGDAIML_Project_Spam_Clustering/datos/los_mensajes_primer_set.csv',encoding='cp1252')

In [ ]:
message_febrero = pd.read_csv('/content/drive/MyDrive/PG_Diploma_AI_ML_2021_UOHYD/PGDAIML_Project_Spam_Clustering/datos/mensajes_febrero.csv',usecols=['CONTENT','STATUS'])

In [ ]:
message_marzo = pd.read_csv('/content/drive/MyDrive/PG_Diploma_AI_ML_2021_UOHYD/PGDAIML_Project_Spam_Clustering/datos/mensajes_marzo.csv',usecols=['CONTENT','STATUS'])

In [ ]:
message_abril = pd.read_csv('/content/drive/MyDrive/PG_Diploma_AI_ML_2021_UOHYD/PGDAIML_Project_Spam_Clustering/datos/mensajes_abril.csv',usecols=['CONTENT','STATUS'])

In [ ]:
message_mayo = pd.read_csv('/content/drive/MyDrive/PG_Diploma_AI_ML_2021_UOHYD/PGDAIML_Project_Spam_Clustering/datos/mensajes_mayo.csv',usecols=['CONTENT','STATUS'])

In [ ]:
messages_de_SC = pd.read_csv('/content/drive/MyDrive/PG_Diploma_AI_ML_2021_UOHYD/PGDAIML_Project_Spam_Clustering/datos/SC_data_5July2019/SC_data_5July2019',sep='\t',usecols=['content','status_id'])

In [ ]:
messages_de_sc_7726 = pd.read_csv('/content/drive/MyDrive/PG_Diploma_AI_ML_2021_UOHYD/PGDAIML_Project_Spam_Clustering/datos/SC_data_5July2019/my_datos_de_ese_ce.csv',usecols=['content','status_id'])

In [ ]:
messages_de_SC['spam_class'] = messages_de_SC[messages_de_SC['status_id'] !=1].status_id.apply( lambda x : "spam" if x==2 or x==4 or x==0  else "clean"  )

In [ ]:
message_mayo['spam_class'] = message_mayo.STATUS.apply( lambda x: "spam" if x=="SFS" or x=="SF" or x=="S" else "clean")

In [ ]:
messages_de_SC.value_counts('spam_class')

In [ ]:
messages_de_SC.head()

### Creating databases, loadings into postgreSQL and exporting as DB files

In [ ]:
datos_para_db = my_data_dict_creations(messages_de_SC)

In [ ]:
with my_psql_engine.connect() as con:

    con.execute(text('DROP TABLE IF EXISTS messages_7726_2'))
    con.execute(text('''CREATE TABLE messages_7726_2( message_id TEXT,
                 message_preview TEXT,message_flag TEXT)'''))

    for mid,msg in datos_para_db:
        con.execute(text("""INSERT INTO messages_7726_2(message_id,message_preview,message_flag) 
            VALUES(:message_id,:content,:spam_class)"""), **mid,**msg)

In [ ]:
my_psql_conexion = my_psql_engine.connect() 

In [ ]:
pd.read_sql('SELECT message_flag,count(*) from messages_7726_2 group by 1',my_psql_conexion)

In [ ]:
inspector = inspect(my_psql_engine)
schemas = inspector.get_schema_names()

In [ ]:
for r in my_psql_conexion.execute(text('SELECT schemaname,relname,n_live_tup FROM pg_stat_user_tables ORDER BY n_live_tup DESC')):
    print(r)

In [ ]:
for r in my_psql_conexion.execute(text('SELECT datname FROM pg_database WHERE datistemplate = false;')):
    print(r)

In [ ]:
# DB EXPORT 
!pg_dump -U postgres -h localhost uohyd_pgdaiml_7726_sc > los_mensajes_class_7726_db

In [ ]:
# DB IMPORTS
!PGPASSWORD=$TFIO_DEMO_DATABASE_PASS psql -q -h $TFIO_DEMO_DATABASE_HOST -p $TFIO_DEMO_DATABASE_PORT -U $TFIO_DEMO_DATABASE_USER -d uohyd_pgdaiml_bkp < my_db_1